# LAB3: Algorytmy grafowe – minimalne drzewo rozpinające grafu

## Zadanie 1

In [3]:
from typing import List, Dict, Tuple, Set
edge = Tuple[int, int]
graph = Dict[int, List]
inf = float('inf')

#implementacja wybranego grafu (nieskierowany, cykliczny, spójny) za pomocą słownika, którego kluczami są wierzchołki, a wartościami lista połączonych wierzchołków
G: graph = { 1: [2,4], 2: [1, 3], 3: [2, 7], 4: [1, 5, 7], 5: [4, 6], 6: [5], 7: [3, 4, 8], 8: [7, 9], 9: [8, 10], 10: [9] }
a = {(1,2): 4, (1,4): 3, (2,3): 1, (3,7): 5, (4,5): 1, (4,7): 1, (5,6): 6, (7,8): 2, (8,9): 3, (9,10): 1}
def DPA(G: graph, a: Dict, s: int) -> Tuple[Set[edge], int]:
    size_graph = len(G) #inicjalziacja 
    beta = [inf for _ in range(size_graph)]
    alfa = [0 for _ in range(size_graph)]
    sum = 0 #inicjalizacja zmiennych: suma krawędzi
    A = set()   #inicjalizacja zmiennych: zbiór krawędzi MST
    Q = list(range(1,size_graph+1))  #inicjalizacja zmiennych: zbiór wierzchołków nienależących do MST
    Q.remove(s) #usunięcie ze zbioru wierzchołków nienależących do MST
    beta[s-1] = 0   #ustawienie wagi początkowego wierzchołka
    u_prev = s #ustawienie s jako poprzedni wierzchołek
    while Q:    #iteracja po wierzchołkach nienależących do MST 
        for u_q in Q: #iteracja po kazdym wierzchołku należącym do Q
            for u in G[u_prev]: #iteracja po kazdym wierzchołku należącym sąsiada wierzchołka u
                if a[u-1][u_prev-1] < beta[u-1]: #jeżeli 
                    alfa[u-1] = u_prev
                    beta[u-1] = a[u-1][u_prev-1]
            
        prev_min = inf
        for u in Q:
            if beta[u-1] < prev_min:
                prev_min = beta[u-1]
                u_prev = u

        Q.remove(u_prev)
        A.add((alfa[u_prev-1], u_prev))
        sum += a[alfa[u_prev-1]-1][u_prev-1]
    return (A, sum)            

In [4]:
graph = { 1: [2, 3, 4, 5], 2: [1, 4, 7], 3: [1, 4, 5, 6], 4: [1, 2, 3, 6, 7], 5: [1, 3, 6], 6: [3, 4, 5, 7], 7: [2, 4, 6] }

a = [[inf, 2, 1, 4, 3, inf, inf], [2, inf, inf, 3, inf, inf, 5], [1, inf, inf, 7, 1, 2, inf], [4, 3, 7, inf, inf, 4, 4], [3, inf, 1, inf, inf, 3, inf], [inf, inf, 2, 4, 3, inf, 3], [inf, 5, inf, 4, inf, 2, inf]]
print(DPA(graph, a, 1))

({(2, 4), (1, 2), (6, 7), (3, 6), (1, 3), (3, 5)}, 12)
